# Workflow Progress Visualization Demo

This notebook demonstrates real-time progress visualization in PyNodeWidget nodes.

## Features:
- **Progress bar field** - Custom field type that displays visual progress
- **Real-time updates** - Python updates sync to frontend automatically
- **Multi-node workflow** - Simulate a data processing pipeline
- **Thread-based execution** - Non-blocking progress updates

In [ ]:
from pydantic import BaseModel, Field
from pynodewidget import NodeFlowWidget, JsonSchemaNodeWidget
import time
import threading

In [ ]:
class DataLoaderParams(BaseModel):
    """Parameters for data loading node."""
    source: str = Field(default="database", description="Data source")
    batch_size: int = Field(default=100, ge=1, le=1000, description="Batch size")
    progress: int = Field(
        default=0,
        ge=0,
        le=100,
        description="Loading progress",
        json_schema_extra={"type": "progress"}
    )


class ProcessorParams(BaseModel):
    """Parameters for data processing node."""
    algorithm: str = Field(default="transform", description="Processing algorithm")
    workers: int = Field(default=4, ge=1, le=16, description="Number of workers")
    progress: int = Field(
        default=0,
        ge=0,
        le=100,
        description="Processing progress",
        json_schema_extra={"type": "progress"}
    )


class AnalyzerParams(BaseModel):
    """Parameters for analysis node."""
    method: str = Field(default="statistical", description="Analysis method")
    confidence: float = Field(default=0.95, ge=0, le=1, description="Confidence level")
    progress: int = Field(
        default=0,
        ge=0,
        le=100,
        description="Analysis progress",
        json_schema_extra={"type": "progress"}
    )


class OutputParams(BaseModel):
    """Parameters for output node."""
    format: str = Field(default="json", description="Output format")
    compress: bool = Field(default=False, description="Compress output")
    progress: int = Field(
        default=0,
        ge=0,
        le=100,
        description="Export progress",
        json_schema_extra={"type": "progress"}
    )

In [ ]:
class DataLoaderNode(JsonSchemaNodeWidget):
    """Load data from source."""
    label = "Data Loader"
    parameters = DataLoaderParams
    icon = "📥"
    description = "Load data from various sources"
    outputs = [{"id": "data", "label": "Data"}]


class ProcessorNode(JsonSchemaNodeWidget):
    """Process data with configurable algorithm."""
    label = "Processor"
    parameters = ProcessorParams
    icon = "⚙️"
    description = "Process data with selected algorithm"
    inputs = [{"id": "input", "label": "Input"}]
    outputs = [{"id": "output", "label": "Output"}]


class AnalyzerNode(JsonSchemaNodeWidget):
    """Analyze processed data."""
    label = "Analyzer"
    parameters = AnalyzerParams
    icon = "📊"
    description = "Perform statistical analysis"
    inputs = [{"id": "data", "label": "Data"}]
    outputs = [{"id": "results", "label": "Results"}]


class OutputNode(JsonSchemaNodeWidget):
    """Export results to file."""
    label = "Output"
    parameters = OutputParams
    icon = "💾"
    description = "Save results to file"
    inputs = [{"id": "results", "label": "Results"}]

In [ ]:
# Create widget with all node types
flow = NodeFlowWidget(
    nodes=[DataLoaderNode, ProcessorNode, AnalyzerNode, OutputNode],
    height="700px"
)

# Create a sample workflow with 4 nodes
flow.nodes = [
    {
        "id": "loader-1",
        "type": "data_loader_node",
        "position": {"x": 50, "y": 100},
        "data": {
            "label": "Data Loader",
            "parameters": flow.node_templates[0]["defaultData"]["parameters"],
            "outputs": [{"id": "data", "label": "Data"}],
            "inputs": [],
            "values": {"source": "database", "batch_size": 100, "progress": 0}
        }
    },
    {
        "id": "processor-1",
        "type": "processor_node",
        "position": {"x": 350, "y": 100},
        "data": {
            "label": "Processor",
            "parameters": flow.node_templates[1]["defaultData"]["parameters"],
            "inputs": [{"id": "input", "label": "Input"}],
            "outputs": [{"id": "output", "label": "Output"}],
            "values": {"algorithm": "transform", "workers": 4, "progress": 0}
        }
    },
    {
        "id": "analyzer-1",
        "type": "analyzer_node",
        "position": {"x": 650, "y": 100},
        "data": {
            "label": "Analyzer",
            "parameters": flow.node_templates[2]["defaultData"]["parameters"],
            "inputs": [{"id": "data", "label": "Data"}],
            "outputs": [{"id": "results", "label": "Results"}],
            "values": {"method": "statistical", "confidence": 0.95, "progress": 0}
        }
    },
    {
        "id": "output-1",
        "type": "output_node",
        "position": {"x": 950, "y": 100},
        "data": {
            "label": "Output",
            "parameters": flow.node_templates[3]["defaultData"]["parameters"],
            "inputs": [{"id": "results", "label": "Results"}],
            "outputs": [],
            "values": {"format": "json", "compress": False, "progress": 0}
        }
    }
]

# Connect the nodes
flow.edges = [
    {
        "id": "e1-2",
        "source": "loader-1",
        "target": "processor-1",
        "sourceHandle": "data",
        "targetHandle": "input"
    },
    {
        "id": "e2-3",
        "source": "processor-1",
        "target": "analyzer-1",
        "sourceHandle": "output",
        "targetHandle": "data"
    },
    {
        "id": "e3-4",
        "source": "analyzer-1",
        "target": "output-1",
        "sourceHandle": "results",
        "targetHandle": "results"
    }
]

flow

In [ ]:
def simulate_node_execution(flow_widget, node_id, duration=2.0, steps=20):
    """
    Simulate node execution by updating progress from 0 to 100.
    
    Args:
        flow_widget: NodeFlowWidget instance
        node_id: ID of the node to update
        duration: Total duration in seconds
        steps: Number of progress updates
    """
    step_duration = duration / steps
    for i in range(steps + 1):
        progress = int((i / steps) * 100)
        flow_widget.update_node_progress(node_id, progress)
        time.sleep(step_duration)


def execute_workflow(flow_widget, sequential=True):
    """
    Execute the workflow by simulating each node's processing.
    
    Args:
        flow_widget: NodeFlowWidget instance
        sequential: If True, execute nodes one by one; if False, execute in parallel
    """
    # Reset all progress to 0
    for node in flow_widget.nodes:
        flow_widget.update_node_progress(node["id"], 0)
    
    print("🚀 Starting workflow execution...")
    
    if sequential:
        # Execute nodes sequentially
        node_ids = ["loader-1", "processor-1", "analyzer-1", "output-1"]
        node_names = ["Data Loader", "Processor", "Analyzer", "Output"]
        
        for node_id, node_name in zip(node_ids, node_names):
            print(f"  ▶️  Executing {node_name}...")
            simulate_node_execution(flow_widget, node_id, duration=2.0, steps=20)
            print(f"  ✅ {node_name} complete")
    else:
        # Execute nodes in parallel (simulating parallel processing)
        threads = []
        for node in flow_widget.nodes:
            thread = threading.Thread(
                target=simulate_node_execution,
                args=(flow_widget, node["id"], 3.0, 30)
            )
            threads.append(thread)
            thread.start()
        
        # Wait for all threads to complete
        for thread in threads:
            thread.join()
    
    print("🎉 Workflow execution complete!")


# Execute the workflow sequentially
execute_workflow(flow, sequential=True)

In [ ]:
def run_workflow_background():
    """Run workflow in background thread."""
    thread = threading.Thread(target=execute_workflow, args=(flow, True))
    thread.start()
    print("🔄 Workflow started in background...")
    return thread

# Start workflow in background
# bg_thread = run_workflow_background()

## Background Execution Example

Run workflow in a background thread to keep the notebook responsive.

In [ ]:
# Display current progress for all nodes
for node in flow.nodes:
    node_id = node["id"]
    label = node["data"]["label"]
    progress = node["data"]["values"].get("progress", 0)
    print(f"{label} ({node_id}): {progress}%")

## Inspect Current State

Check the current progress values of all nodes.

In [ ]:
# Reset all progress to 0
for node in flow.nodes:
    flow.update_node_progress(node["id"], 0)
print("✓ All progress reset to 0")

In [ ]:
# Manually set progress for specific nodes
flow.update_node_progress("loader-1", 75)
flow.update_node_progress("processor-1", 50)
flow.update_node_progress("analyzer-1", 25)
flow.update_node_progress("output-1", 0)

## Manual Progress Control

You can also manually update progress for individual nodes.

In [ ]:
# Execute all nodes in parallel
execute_workflow(flow, sequential=False)

## Execute in Parallel

Run all nodes simultaneously to see parallel progress updates.

## Execute Workflow with Progress Updates

Simulate workflow execution where each node updates its progress in real-time.

## Create the Workflow Widget

Initialize the widget with all node types and create a sample workflow pipeline.

## Define Workflow Nodes with Progress Fields

Each node has a `progress` field with `json_schema_extra={"type": "progress"}` to use the custom progress bar renderer.